# NASA Exoplanet Data Cleaning Pipeline
## Datasets: Kepler, K2, and TESS

This notebook will explore, clean, and potentially combine three NASA exoplanet discovery datasets:
- **Kepler**: Original Kepler mission data
- **K2**: Extended K2 mission data
- **TESS**: Transiting Exoplanet Survey Satellite data

### Analysis Plan:
1. **Setup & Import** - Load required libraries
2. **Data Loading** - Load all three CSV files
3. **Individual Exploration** - Examine each dataset's structure, statistics, and quality
4. **Schema Comparison** - Identify common/unique columns across datasets
5. **Data Cleaning** - Handle missing values, duplicates, and inconsistencies
6. **Combination Strategy** - Determine if/how to combine datasets
7. **Export** - Save cleaned data for analysis

## 1. Setup & Imports

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configure display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Load Datasets

Load all three NASA exoplanet datasets from the raw data folder.

In [8]:
# Define data paths
data_dir = Path('../data/raw')

# Load datasets
# NASA Exoplanet Archive files use '#' for comment lines
print("Loading datasets...")
kepler_df = pd.read_csv(data_dir / 'kepler.csv', comment='#', on_bad_lines='skip')
k2_df = pd.read_csv(data_dir / 'k2.csv', comment='#', on_bad_lines='skip')
tess_df = pd.read_csv(data_dir / 'tess.csv', comment='#', on_bad_lines='skip')

print(f"✓ Kepler dataset loaded: {kepler_df.shape[0]:,} rows × {kepler_df.shape[1]} columns")
print(f"✓ K2 dataset loaded: {k2_df.shape[0]:,} rows × {k2_df.shape[1]} columns")
print(f"✓ TESS dataset loaded: {tess_df.shape[0]:,} rows × {tess_df.shape[1]} columns")

Loading datasets...
✓ Kepler dataset loaded: 9,564 rows × 141 columns
✓ K2 dataset loaded: 4,004 rows × 295 columns
✓ TESS dataset loaded: 7,703 rows × 87 columns
✓ Kepler dataset loaded: 9,564 rows × 141 columns
✓ K2 dataset loaded: 4,004 rows × 295 columns
✓ TESS dataset loaded: 7,703 rows × 87 columns


## 3. Explore Individual Datasets

### 3.1 Kepler Dataset Exploration

In [14]:
# Kepler: Basic information
print("=" * 80)
print("KEPLER DATASET EXPLORATION")
print("=" * 80)

print(f"\nDataset shape: {kepler_df.shape[0]:,} rows × {kepler_df.shape[1]} columns\n")

print("Column names and types:")
print(kepler_df.dtypes)

print("\n" + "-" * 80)
print("First few rows:")
display(kepler_df.head())

print("\n" + "-" * 80)
print("Missing values:")
missing_kepler = kepler_df.isnull().sum()
missing_pct_kepler = (missing_kepler / len(kepler_df) * 100).round(2)
missing_df_kepler = pd.DataFrame({
    'Missing Count': missing_kepler[missing_kepler > 0],
    'Percentage': missing_pct_kepler[missing_kepler > 0]
}).sort_values('Percentage', ascending=False)
display(missing_df_kepler)

print("\n" + "-" * 80)
print("Basic statistics:")
display(kepler_df.describe())

KEPLER DATASET EXPLORATION

Dataset shape: 9,564 rows × 141 columns

Column names and types:
rowid                   int64
kepid                   int64
kepoi_name             object
kepler_name            object
koi_disposition        object
                       ...   
koi_dikco_mra_err     float64
koi_dikco_mdec        float64
koi_dikco_mdec_err    float64
koi_dikco_msky        float64
koi_dikco_msky_err    float64
Length: 141, dtype: object

--------------------------------------------------------------------------------
First few rows:


,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_disp_prov,koi_comment,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_eccen_err1,koi_eccen_err2,koi_longp,koi_longp_err1,koi_longp_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_ingress,koi_ingress_err1,koi_ingress_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_fittype,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_sma_err1,koi_sma_err2,koi_incl,koi_incl_err1,koi_incl_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_limbdark_mod,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_parm_prov,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_tce_delivname,koi_quarters,koi_bin_oedp_sig,koi_trans_mod,koi_model_dof,koi_model_chisq,koi_datalink_dvr,koi_datalink_dvs,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sage,koi_sage_err1,koi_sage_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,9.488,0.000,-0.000,170.539,0.002,-0.002,2455003.539,0.002,-0.002,0.000,NaN,NaN,NaN,NaN,NaN,0.146,0.318,-0.146,2.958,0.082,-0.082,NaN,NaN,NaN,615.800,19.500,-19.500,0.022,0.001,-0.001,3.208,0.332,-1.100,LS+MCMC,2.260,0.260,-0.150,0.085,NaN,NaN,89.660,NaN,NaN,793.000,NaN,NaN,93.590,29.450,-16.650,24.810,2.600,-2.600,Claret (2011 A&A 529 75) ATLAS LS,0.000,0.000,0.229,0.460,q1_q17_dr25_koi,5.136,28.471,35.800,2,142.000,1.000,q1_q17_dr25_tce,11111111111111111000000000000000,0.686,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-001-2016...,5455.000,81.000,-81.000,4.467,0.064,-0.096,0.140,0.150,-0.150,0.927,0.105,-0.061,0.919,0.052,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.934,48.142,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.002,19.462,0.000,48.142,0.000,0.430,0.510,0.940,0.480,-0.000,0.000,-0.001,0.000,-0.010,0.130,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,54.418,0.000,-0.000,162.514,0.004,-0.004,2454995.514,0.004,-0.004,0.000,NaN,NaN,NaN,NaN,NaN,0.586,0.059,-0.443,4.507,0.116,-0.116,NaN,NaN,NaN,874.800,35.500,-35.500,0.028,0.009,-0.001,3.024,2.205,-2.496,LS+MCMC,2.830,0.320,-0.190,0.273,NaN,NaN,89.570,NaN,NaN,443.000,NaN,NaN,9.110,2.870,-1.620,77.900,28.400,-28.400,Claret (2011 A&A 529 75) ATLAS LS,0.000,0.000,0.229,0.460,q1_q17_dr25_koi,7.028,20.110,25.800,2,25.000,2.000,q1_q17_dr25_tce,11111111111111111000000000000000,0.002,Mandel and Agol (2002 ApJ 580 171),NaN,NaN,010/010797/010797460/dv/kplr010797460-20160209...,010/010797/010797460/dv/kplr010797460-002-2016...,5455.000,81.000,-81.000,4.467,0.064,-0.096,0.140,0.150,-0.150,0.927,0.105,-0.061,0.919,0.052,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.934,48.142,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.003,19.462,0.000,48.142,0.000,-0.630,0.720,1.230,0.


--------------------------------------------------------------------------------
Missing values:


,Missing Count,Percentage
koi_ingress_err2,9564,100.000
koi_longp_err2,9564,100.000
koi_teq_err1,9564,100.000
koi_incl_err2,9564,100.000
koi_incl_err1,9564,100.000
...,...,...
koi_kmag,25,0.260
koi_hmag,25,0.260
koi_jmag,25,0.260
koi_rmag,9,0.090



--------------------------------------------------------------------------------
Basic statistics:


,rowid,kepid,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_eccen_err1,koi_eccen_err2,koi_longp,koi_longp_err1,koi_longp_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_ingress,koi_ingress_err1,koi_ingress_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_ror,koi_ror_err1,koi_ror_err2,koi_srho,koi_srho_err1,koi_srho_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_sma,koi_sma_err1,koi_sma_err2,koi_incl,koi_incl_err1,koi_incl_err2,koi_teq,koi_teq_err1,koi_teq_err2,koi_insol,koi_insol_err1,koi_insol_err2,koi_dor,koi_dor_err1,koi_dor_err2,koi_ldm_coeff4,koi_ldm_coeff3,koi_ldm_coeff2,koi_ldm_coeff1,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_count,koi_num_transits,koi_tce_plnt_num,koi_bin_oedp_sig,koi_model_dof,koi_model_chisq,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_smet,koi_smet_err1,koi_smet_err2,koi_srad,koi_srad_err1,koi_srad_err2,koi_smass,koi_smass_err1,koi_smass_err2,koi_sage,koi_sage_err1,koi_sage_err2,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
count,9564.000,9564.000,8054.000,9564.000,9564.000,9564.000,9564.000,9564.000,9110.000,9110.000,9564.000,9110.000,9110.000,9564.000,9110.000,9110.000,9201.000,0.000,0.000,0.000,0.000,0.000,9201.000,9110.000,9110.000,9564.000,9110.000,9110.000,0.000,0.000,0.000,9201.000,9110.000,9110.000,9201.000,9201.000,9201.000,9243.000,9243.000,9243.000,9201.000,9201.000,9201.000,9201.000,0.000,0.000,9200.000,0.000,0.000,9201.000,0.000,0.000,9243.000,9243.000,9243.000,9201.000,9110.000,9110.000,9201.000,9201.000,9201.000,9201.000,8422.000,8422.000,9201.000,9564.000,8422.000,9218.000,8054.000,0.000,0.000,9201.000,9096.000,9081.000,9201.000,9096.000,9096.000,9178.000,9177.000,9177.000,9201.000,9096.000,9096.000,9201.000,9096.000,9096.000,0.000,0.000,0.000,9564.000,9564.000,9563.000,9523.000,9555.000,9410.000,8951.000,9539.000,9539.000,9539.000,8488.000,9058.000,9058.000,9058.000,9058.000,9109.000,9109.000,9109.000,9109.000,8734.000,8734.000,8747.000,8747.000,8965.000,8965.000,8965.000,8965.000,8965.000,8965.000,8994.000,8994.000,8994.000,8994.000,8994.000,8994.000
mean,4782.500,7690628.327,0.481,0.209,0.233,0.198,0.120,75.671,0.002,-0.002,166.183,0.010,-0.010,2454999.183,0.010,-0.010,0.000,NaN,NaN,NaN,NaN,NaN,0.735,1.960,-0.333,5.622,0.340,-0.340,NaN,NaN,NaN,23791.336,123.198,-123.198,0.284,1.782,-0.101,9.164,18.065,-5.489,102.892,17.658,-33.023,0.224,NaN,NaN,82.469,NaN,NaN,1085.386,NaN,NaN,7745.737,3750.698,-4043.522,76.736,23.679,-23.679,0.000,0.000,0.254,0.408,176.846,1025.665,259.895,1.406,385.007,1.244,0.409,NaN,NaN,5706.823,144.636,-162.265,4.310,0.121,-0.143,-0.124,0.229,-0.252,1.729,0.362,-0.395,1.024,0.123,-0.139,NaN,NaN,NaN,292.060,43.810,14.265,14.831,14.222,14.075,13.992,12.993,12.621,12.543,0.151,19.471,0.000,43.829,0.000,-0.316,0.704,-0.166,0.706,-0.000,0.222,-0.001,0.309,-0.012,0.434,-0.045,0.446,1.867,0.490,-0.024,0.425,-0.077,0.437,1.813,0.476
std,2761.033,2653459.081,0.477,4.767,0.423,0.398,0.325,1334.744,0.008,0.008,67.919,0.023,0.023,67.919,0.023,0.023,0.000,NaN,NaN,NaN,NaN,NaN,3.349,9.422,1.250,6.472,0.670,0.670,NaN,NaN,NaN,82242.683,4112.615,4112.615,3.307,9.407,1.241,53.808,76.801,32.337,3077.639,391.139,1193.520,0.566,NaN,NaN,15.224,NaN,NaN,856.351,NaN,NaN,159204.665,55044.209,88388.311,845.275,298.215,298.215,0.000,0.000,0.065,0.106,770.902,4154.122,795.807,0.873,545.756,0.665,0.501,NaN,N

### 3.2 K2 Dataset Exploration

In [15]:
# K2: Basic information
print("=" * 80)
print("K2 DATASET EXPLORATION")
print("=" * 80)

print(f"\nDataset shape: {k2_df.shape[0]:,} rows × {k2_df.shape[1]} columns\n")

print("Column names and types:")
print(k2_df.dtypes)

print("\n" + "-" * 80)
print("First few rows:")
display(k2_df.head())

print("\n" + "-" * 80)
print("Missing values:")
missing_k2 = k2_df.isnull().sum()
missing_pct_k2 = (missing_k2 / len(k2_df) * 100).round(2)
missing_df_k2 = pd.DataFrame({
    'Missing Count': missing_k2[missing_k2 > 0],
    'Percentage': missing_pct_k2[missing_k2 > 0]
}).sort_values('Percentage', ascending=False)
display(missing_df_k2)

print("\n" + "-" * 80)
print("Basic statistics:")
display(k2_df.describe())

K2 DATASET EXPLORATION

Dataset shape: 4,004 rows × 295 columns

Column names and types:
rowid             int64
pl_name          object
hostname         object
pl_letter        object
k2_name          object
                 ...   
st_nrvc         float64
st_nspec        float64
pl_nespec       float64
pl_ntranspec    float64
pl_ndispec      float64
Length: 295, dtype: object

--------------------------------------------------------------------------------
First few rows:


,rowid,pl_name,hostname,pl_letter,k2_name,epic_hostname,epic_candname,hd_name,hip_name,tic_id,gaia_id,default_flag,disposition,disp_refname,sy_snum,sy_pnum,sy_mnum,cb_flag,discoverymethod,disc_year,disc_refname,disc_pubdate,disc_locale,disc_facility,disc_telescope,disc_instrument,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_masse,pl_masseerr1,pl_masseerr2,pl_masselim,pl_massj,pl_massjerr1,pl_massjerr2,pl_massjlim,pl_msinie,pl_msinieerr1,pl_msinieerr2,pl_msinielim,pl_msinij,pl_msinijerr1,pl_msinijerr2,pl_msinijlim,pl_cmasse,pl_cmasseerr1,pl_cmasseerr2,pl_cmasselim,pl_cmassj,pl_cmassjerr1,pl_cmassjerr2,pl_cmassjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_tsystemref,ttv_flag,pl_imppar,pl_impparerr1,pl_impparerr2,pl_impparlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandur,pl_trandurerr1,pl_trandurerr2,pl_trandurlim,pl_ratdor,pl_ratdorerr1,pl_ratdorerr2,pl_ratdorlim,pl_ratror,pl_ratrorerr1,pl_ratrorerr2,pl_ratrorlim,pl_occdep,pl_occdeperr1,pl_occdeperr2,pl_occdeplim,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtperlim,pl_orblper,pl_orblpererr1,pl_orblpererr2,pl_orblperlim,pl_rvamp,pl_rvamperr1,pl_rvamperr2,pl_rvamplim,pl_projobliq,pl_projobliqerr1,pl_projobliqerr2,pl_projobliqlim,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2,pl_trueobliqlim,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,sy_refname,rastr,ra,decstr,dec,glat,glon,elat,elon,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmdec,sy_pmdecerr1,sy_pmdecerr2,sy_dist,sy_disterr1,sy_disterr2,sy_plx,sy_plxerr1,sy_plxerr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_umag,sy_umagerr1,sy_umagerr2,sy_gmag,sy_gmagerr1,sy_gmagerr2,sy_rmag,sy_rmagerr1,sy_rmagerr2,sy_imag,sy_imagerr1,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,k2_campaigns,k2_campaigns_num,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec
0,1,BD+20 594 b,BD+20 594,b,K2-56 b,EPIC 210848071,EPIC 210848071.01,NaN,NaN,TIC 26123781,Gaia DR2 58200934326315136,0,CONFIRMED,Espinoza et al. 2016,1.000,1.000,0.000,0.000,Transit,2016.000,<a refstr=ESPINOZA_ET_AL__2016 href=https://ui...,2016-10,Space,K2,0.95 m Kepler Telescope,Kepler CCD Array,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,Published Confirmed,0.000,<a refstr=MAYO_ET_AL__2018 href=https://ui.ads...,41.689,0.003,-0.003,0.000,NaN,NaN,NaN,NaN,2.355,0.310,-0.167,0.000,0.210,0.028,-0.015,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


--------------------------------------------------------------------------------
Missing values:


,Missing Count,Percentage
sy_kepmagerr2,4004,100.000
sy_kepmagerr1,4004,100.000
sy_icmagerr2,4004,100.000
sy_icmagerr1,4004,100.000
sy_icmag,4004,100.000
...,...,...
disc_year,17,0.420
discoverymethod,17,0.420
rv_flag,17,0.420
tic_id,4,0.100



--------------------------------------------------------------------------------
Basic statistics:


,rowid,default_flag,sy_snum,sy_pnum,sy_mnum,cb_flag,disc_year,rv_flag,pul_flag,ptv_flag,tran_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,dkin_flag,pl_controv_flag,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_masse,pl_masseerr1,pl_masseerr2,pl_masselim,pl_massj,pl_massjerr1,pl_massjerr2,pl_massjlim,pl_msinie,pl_msinieerr1,pl_msinieerr2,pl_msinielim,pl_msinij,pl_msinijerr1,pl_msinijerr2,pl_msinijlim,pl_cmasse,pl_cmasseerr1,pl_cmasseerr2,pl_cmasselim,pl_cmassj,pl_cmassjerr1,pl_cmassjerr2,pl_cmassjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,ttv_flag,pl_imppar,pl_impparerr1,pl_impparerr2,pl_impparlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandur,pl_trandurerr1,pl_trandurerr2,pl_trandurlim,pl_ratdor,pl_ratdorerr1,pl_ratdorerr2,pl_ratdorlim,pl_ratror,pl_ratrorerr1,pl_ratrorerr2,pl_ratrorlim,pl_occdep,pl_occdeperr1,pl_occdeperr2,pl_occdeplim,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtperlim,pl_orblper,pl_orblpererr1,pl_orblpererr2,pl_orblperlim,pl_rvamp,pl_rvamperr1,pl_rvamperr2,pl_rvamplim,pl_projobliq,pl_projobliqerr1,pl_projobliqerr2,pl_projobliqlim,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2,pl_trueobliqlim,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_age,st_ageerr1,st_ageerr2,st_agelim,st_dens,st_denserr1,st_denserr2,st_denslim,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_radv,st_radverr1,st_radverr2,st_radvlim,ra,dec,glat,glon,elat,elon,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmdec,sy_pmdecerr1,sy_pmdecerr2,sy_dist,sy_disterr1,sy_disterr2,sy_plx,sy_plxerr1,sy_plxerr2,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_umag,sy_umagerr1,sy_umagerr2,sy_gmag,sy_gmagerr1,sy_gmagerr2,sy_rmag,sy_rmagerr1,sy_rmagerr2,sy_imag,sy_imagerr1,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,pl_nnotes,k2_campaigns_num,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_ndispec
count,4004.000,4004.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3987.000,3937.000,3052.000,3052.000,3937.000,812.000,805.000,805.000,812.000,3159.000,2874.000,2874.000,3159.000,3159.000,2874.000,2874.000,3159.000,408.000,366.000,366.000,408.000,408.000,366.000,366.000,408.000,31.000,31.000,31.000,31.000,31.000,31.000,31.000,31.000,15.000,15.000,15.000,15.000,15.000,15.000,15.000,15.000,428.000,386.000,386.000,428.000,428.000,386.000,386.000,428.000,352.000,327.000,327.000,352.000,422.000,226.000,226.000,422.000,629.000,447.000,447.000,629.000,845.000,677.000,677.000,845.000,987.000,945.000,945.000,987.000,3916.000,3019.000,3019.000,3916.000,3981.000,1478.000,1076.000,1076.000,1478.000,2085.000,1295.000,1295.000,2085.000,2767.000,1857.000,1854.000,2767.000,2191.000,2151.000,2151.000,2191.000,3004.000,2603.000,2603.000,3004.000,1.000,0.000,0.000,1.000,31.000,30.000,30.000,31.000,246.000,215.000,2

### 3.3 TESS Dataset Exploration

In [16]:
# TESS: Basic information
print("=" * 80)
print("TESS DATASET EXPLORATION")
print("=" * 80)

print(f"\nDataset shape: {tess_df.shape[0]:,} rows × {tess_df.shape[1]} columns\n")

print("Column names and types:")
print(tess_df.dtypes)

print("\n" + "-" * 80)
print("First few rows:")
display(tess_df.head())

print("\n" + "-" * 80)
print("Missing values:")
missing_tess = tess_df.isnull().sum()
missing_pct_tess = (missing_tess / len(tess_df) * 100).round(2)
missing_df_tess = pd.DataFrame({
    'Missing Count': missing_tess[missing_tess > 0],
    'Percentage': missing_pct_tess[missing_tess > 0]
}).sort_values('Percentage', ascending=False)
display(missing_df_tess)

print("\n" + "-" * 80)
print("Basic statistics:")
display(tess_df.describe())

TESS DATASET EXPLORATION

Dataset shape: 7,703 rows × 87 columns

Column names and types:
rowid                  int64
toi                  float64
toipfx                 int64
tid                    int64
ctoi_alias           float64
pl_pnum                int64
tfopwg_disp           object
rastr                 object
ra                   float64
raerr1               float64
raerr2               float64
decstr                object
dec                  float64
decerr1              float64
decerr2              float64
st_pmra              float64
st_pmraerr1          float64
st_pmraerr2          float64
st_pmralim           float64
st_pmrasymerr        float64
st_pmdec             float64
st_pmdecerr1         float64
st_pmdecerr2         float64
st_pmdeclim          float64
st_pmdecsymerr       float64
pl_tranmid           float64
pl_tranmiderr1       float64
pl_tranmiderr2       float64
pl_tranmidlim          int64
pl_tranmidsymerr       int64
pl_orbper            float64
pl_orbperer

,rowid,toi,toipfx,tid,ctoi_alias,pl_pnum,tfopwg_disp,rastr,ra,raerr1,raerr2,decstr,dec,decerr1,decerr2,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmrasymerr,st_pmdec,st_pmdecerr1,st_pmdecerr2,st_pmdeclim,st_pmdecsymerr,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_tranmidsymerr,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbpersymerr,pl_trandurh,pl_trandurherr1,pl_trandurherr2,pl_trandurhlim,pl_trandurhsymerr,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandepsymerr,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radesymerr,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_insolsymerr,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_eqtsymerr,st_tmag,st_tmagerr1,st_tmagerr2,st_tmaglim,st_tmagsymerr,st_dist,st_disterr1,st_disterr2,st_distlim,st_distsymerr,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_teffsymerr,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_loggsymerr,st_rad,st_raderr1,st_raderr2,st_radlim,st_radsymerr,toi_created,rowupdate
0,1,1000.010,1000,50365310,50365310.010,1,FP,07h29m25.85s,112.358,NaN,NaN,-12d41m45.46s,-12.696,NaN,NaN,-5.964,0.085,-0.085,0.000,1.000,-0.076,0.072,-0.072,0.000,1.000,2459229.630,0.002,-0.002,0,1,2.171,0.000,-0.000,0,1,2.017,0.320,-0.320,0,1,656.886,37.778,-37.778,0,1,5.818,1.911,-1.911,0,1,22601.949,NaN,NaN,NaN,NaN,3127.204,NaN,NaN,NaN,NaN,9.604,0.013,-0.013,0,1,485.735,11.951,-11.951,0,1,10249.000,264.700,-264.700,0,1,4.190,0.070,-0.070,0,1,2.170,0.073,-0.073,0,1,2019-07-24 15:58:33,2024-09-09 10:08:01
1,2,1001.010,1001,88863718,88863718.010,1,PC,08h10m19.31s,122.580,NaN,NaN,-05d30m49.87s,-5.514,NaN,NaN,-4.956,0.102,-0.102,0.000,1.000,-15.555,0.072,-0.072,0.000,1.000,2459987.949,0.002,-0.002,0,1,1.932,0.000,-0.000,0,1,3.166,0.647,-0.647,0,1,1286.000,1186.490,-1186.490,0,1,11.215,2.624,-2.624,0,1,44464.500,NaN,NaN,NaN,NaN,4045.000,NaN,NaN,NaN,NaN,9.423,0.006,-0.006,0,1,295.862,5.910,-5.910,0,1,7070.000,126.400,-126.400,0,1,4.030,0.090,-0.090,0,1,2.010,0.090,-0.090,0,1,2019-07-24 15:58:33,2023-04-03 14:31:04
2,3,1002.010,1002,124709665,124709665.010,1,FP,06h58m54.47s,104.727,NaN,NaN,-10d34m49.64s,-10.580,NaN,NaN,-1.462,0.206,-0.206,0.000,1.000,-2.249,0.206,-0.206,0.000,1.000,2459224.688,0.001,-0.001,0,1,1.868,0.000,-0.000,0,1,1.408,0.184,-0.184,0,1,1500.000,1.758,-1.758,0,1,23.753,NaN,NaN,0,1,2860.610,NaN,NaN,NaN,NaN,2037.000,NaN,NaN,NaN,NaN,9.300,0.058,-0.058,0,1,943.109,106.333,-106.333,0,1,8924.000,124.000,-124.000,0,1,NaN,NaN,NaN,0,1,5.730,NaN,NaN,0,1,2019-07-24 15:58:33,2022-07-11 16:02:02
3,4,1003.010,1003,106997505,106997505.010,1,FP,07h22m14.39s,110.560,NaN,NaN,-25d12m25.26s,-25.207,NaN,NaN,-0.939,0.041,-0.041,0.000,1.000,1.640,0.055,-0.055,0.000,1.000,2458493.396,0.005,-0.005,0,1,2.743,0.001,-0.001,0,1,3.167,0.642,-0.642,0,1,383.410,0.782,-0.782,0,1,NaN,NaN,NaN,0,1,1177.360,NaN,NaN,NaN,NaN,1631.000,NaN,NaN,NaN,NaN,9.300,0.037,-0.037,0,1,7728.170,1899.570,-1899.570,0,1,5388.500,567.000,-567.000,0,1,4.150,1.640,-1.640,0,1,NaN,NaN,NaN,0,1,2019-07-24 15:58:33,2022-02-23 10:10:02
4,5,1004.010,1004,238597883,238597883.010,1,FP,08h08m42.77s,122.178,NaN,NaN,-48d48m10.12s,-48.803,NaN,NaN,-4.496,0.069,-0.069,0.000,1.000,9.347,0.062,-0.062,0.000,1.000,2459987.047,0.004,-0.004,0,1,3.573,0.000,-0.000,0,1,3.370,1.029,-1.029,0,1,755.000,1306.550,-1306.550,0,1,11.311,3.247,-3.247,0,1,54679.300,NaN,NaN,NaN,NaN,4260.000,NaN,NaN,NaN,NaN,9.136,0.006,-0.006,0,1,356.437,4.617,-4.617,0,1,9219.000,171.100,-171.100,0,1,4.140,0.070,-0.070,0,1,2.150,0.060,-0.060,0,1,2019-07-24 15:58:33,2024-09-09 10:08:01



--------------------------------------------------------------------------------
Missing values:


,Missing Count,Percentage
raerr1,7703,100.000
raerr2,7703,100.000
decerr1,7703,100.000
decerr2,7703,100.000
pl_eqtsymerr,7703,100.000
pl_eqtlim,7703,100.000
pl_eqterr2,7703,100.000
pl_eqterr1,7703,100.000
pl_insolsymerr,7703,100.000
pl_insollim,7703,100.000



--------------------------------------------------------------------------------
Basic statistics:


,rowid,toi,toipfx,tid,ctoi_alias,pl_pnum,ra,raerr1,raerr2,dec,decerr1,decerr2,st_pmra,st_pmraerr1,st_pmraerr2,st_pmralim,st_pmrasymerr,st_pmdec,st_pmdecerr1,st_pmdecerr2,st_pmdeclim,st_pmdecsymerr,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_tranmidsymerr,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbpersymerr,pl_trandurh,pl_trandurherr1,pl_trandurherr2,pl_trandurhlim,pl_trandurhsymerr,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandepsymerr,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radesymerr,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_insolsymerr,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_eqtsymerr,st_tmag,st_tmagerr1,st_tmagerr2,st_tmaglim,st_tmagsymerr,st_dist,st_disterr1,st_disterr2,st_distlim,st_distsymerr,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_teffsymerr,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_loggsymerr,st_rad,st_raderr1,st_raderr2,st_radlim,st_radsymerr
count,7703.000,7703.000,7703.000,7703.000,7703.000,7703.000,7703.000,0.000,0.000,7703.000,0.000,0.000,7569.000,7569.000,7569.000,7569.000,7569.000,7569.000,7569.000,7569.000,7569.000,7569.000,7703.000,7692.000,7692.000,7703.000,7703.000,7596.000,7572.000,7572.000,7703.000,7703.000,7703.000,7690.000,7690.000,7703.000,7703.000,7703.000,7697.000,7697.000,7703.000,7703.000,7197.000,6080.000,6080.000,7703.000,7703.000,7527.000,0.000,0.000,0.000,0.000,7392.000,0.000,0.000,0.000,0.000,7703.000,7703.000,7703.000,7703.000,7703.000,7488.000,6996.000,6996.000,7703.000,7703.000,7542.000,7229.000,7229.000,7703.000,7703.000,6847.000,5432.000,5432.000,7703.000,7703.000,7196.000,5740.000,5740.000,7703.000,7703.000
mean,3852.000,3749.039,3749.029,245522652.752,245522652.762,1.049,179.804,NaN,NaN,1.157,NaN,NaN,-0.567,0.229,-0.229,0.000,1.000,-9.182,0.223,-0.223,0.000,1.000,2459552.363,0.004,-0.004,0.000,1.000,17.740,0.000,-0.000,0.000,1.000,3.059,0.362,-0.362,0.000,1.000,8256.683,489.867,-489.867,0.000,1.000,10.327,1.459,-1.459,0.000,1.000,2246.183,NaN,NaN,NaN,NaN,1282.691,NaN,NaN,NaN,NaN,11.564,0.010,-0.010,0.000,1.000,478.223,19.529,-19.529,0.000,1.000,5791.481,205.574,-205.574,0.000,1.000,4.305,0.175,-0.175,0.000,1.000,1.404,0.074,-0.074,0.000,1.000
std,2223.809,2152.357,2152.358,161696778.613,161696778.613,0.273,103.706,NaN,NaN,47.465,NaN,NaN,77.004,0.632,0.632,0.000,0.000,66.875,0.623,0.623,0.000,0.000,615.505,0.044,0.044,0.000,0.000,97.737,0.001,0.001,0.000,0.000,1.874,1.932,1.932,0.000,0.000,17502.276,2148.259,2148.259,0.000,0.000,8.527,3.747,3.747,0.000,0.000,10910.072,NaN,NaN,NaN,NaN,686.724,NaN,NaN,NaN,NaN,1.632,0.033,0.033,0.000,0.000,557.914,137.239,137.239,0.000,0.000,1481.355,550.050,550.050,0.000,0.000,0.304,0.352,0.352,0.000,0.000,1.598,0.082,0.082,0.000,0.000
min,1.000,101.010,101.000,2876.000,2876.010,1.000,0.085,NaN,NaN,-89.472,NaN,NaN,-1624.050,0.015,-8.000,0.000,1.000,-1230.620,0.016,-8.000,0.000,1.000,2457925.865,0.000,-3.305,0.000,1.000,0.152,0.000,-0.022,0.000,1.000,0.101,0.001,-167.615,0.000,1.000,24.583,0.013,-103577.016,0.000,1.000,0.553,0.004,-163.506,0.000,1.000,0.000,NaN,NaN,NaN,NaN,37.000,NaN,NaN,NaN,NaN,4.628,0.000,-1.000,0.000,1.000,6.531,0.003,-4020.600,0.000,1.000,2808.000,4.400,-7000.000,0.000,1.000,0.100,0.000,-2.011,0.000,1.000,0.115,0.003,-1.723,0.000,1.000
25%,1926.500,1861.510,1861.500,131138867.000,131138867.010,1.000,96.070,NaN,NaN,-43.355,NaN,NaN,-10.617,0.038,-0.080,0.000,1.000,-14.695,0.038,-0.071,0.000,1.000,2459196.571,0.001,-0.003,0.000,1.000,2.491,0.000,-0.000,0.000,1.000,1.846,0.139,-0.441,0.000,1.000,1418.165,9.302,-299.146,0.000,1.000,4.496,0.437,-1.108,0.000,1.000,85.293,NaN,NaN,NaN,NaN,813.759,NaN,NaN,NaN,NaN,10.394,0.006,-0.007,0.000,1.000,178.464,1.012,-12.072,0.000,1.000,5211.000,122.000,-157.000,0.000,1.000,4.124,0.080,-0.093,0.000,1.000,0.890,0.050,-0.080,0.000,1.000
50%,3852.000,3736.010,3736.000,249945230.000,249945230.010,1.000,161.157,NaN,NaN,4.715,NaN,NaN,-1.571,0.051,-0.051,0.000,1.000,-3.469,0.049,-0.049,0.000,1.000,2459585.368,0.002,-0.002,0.000,1.000,4.089,0.

## 4. Compare Dataset Schemas

Analyze columns across all three datasets to identify:
- Common columns (appear in all datasets)
- Shared columns (appear in 2 datasets)
- Unique columns (appear in only 1 dataset)

In [17]:
# Get column sets
kepler_cols = set(kepler_df.columns)
k2_cols = set(k2_df.columns)
tess_cols = set(tess_df.columns)

# Find common and unique columns
common_all = kepler_cols & k2_cols & tess_cols
kepler_k2_only = (kepler_cols & k2_cols) - tess_cols
kepler_tess_only = (kepler_cols & tess_cols) - k2_cols
k2_tess_only = (k2_cols & tess_cols) - kepler_cols
kepler_unique = kepler_cols - k2_cols - tess_cols
k2_unique = k2_cols - kepler_cols - tess_cols
tess_unique = tess_cols - kepler_cols - k2_cols

print("=" * 80)
print("SCHEMA COMPARISON")
print("=" * 80)

print(f"\n📊 Total columns:")
print(f"  - Kepler: {len(kepler_cols)}")
print(f"  - K2: {len(k2_cols)}")
print(f"  - TESS: {len(tess_cols)}")

print(f"\n✅ Common to ALL three datasets ({len(common_all)} columns):")
if common_all:
    for col in sorted(common_all):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔄 Shared between Kepler & K2 only ({len(kepler_k2_only)} columns):")
if kepler_k2_only:
    for col in sorted(kepler_k2_only):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔄 Shared between Kepler & TESS only ({len(kepler_tess_only)} columns):")
if kepler_tess_only:
    for col in sorted(kepler_tess_only):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔄 Shared between K2 & TESS only ({len(k2_tess_only)} columns):")
if k2_tess_only:
    for col in sorted(k2_tess_only):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔸 Unique to Kepler ({len(kepler_unique)} columns):")
if kepler_unique:
    for col in sorted(kepler_unique):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔸 Unique to K2 ({len(k2_unique)} columns):")
if k2_unique:
    for col in sorted(k2_unique):
        print(f"  - {col}")
else:
    print("  (none)")

print(f"\n🔸 Unique to TESS ({len(tess_unique)} columns):")
if tess_unique:
    for col in sorted(tess_unique):
        print(f"  - {col}")
else:
    print("  (none)")

SCHEMA COMPARISON

📊 Total columns:
  - Kepler: 141
  - K2: 295
  - TESS: 87

✅ Common to ALL three datasets (3 columns):
  - dec
  - ra
  - rowid

🔄 Shared between Kepler & K2 only (0 columns):
  (none)

🔄 Shared between Kepler & TESS only (0 columns):
  (none)

🔄 Shared between K2 & TESS only (39 columns):
  - decstr
  - pl_eqt
  - pl_eqterr1
  - pl_eqterr2
  - pl_eqtlim
  - pl_insol
  - pl_insolerr1
  - pl_insolerr2
  - pl_insollim
  - pl_orbper
  - pl_orbpererr1
  - pl_orbpererr2
  - pl_orbperlim
  - pl_rade
  - pl_radeerr1
  - pl_radeerr2
  - pl_radelim
  - pl_trandep
  - pl_trandeperr1
  - pl_trandeperr2
  - pl_trandeplim
  - pl_tranmid
  - pl_tranmiderr1
  - pl_tranmiderr2
  - pl_tranmidlim
  - rastr
  - rowupdate
  - st_logg
  - st_loggerr1
  - st_loggerr2
  - st_logglim
  - st_rad
  - st_raderr1
  - st_raderr2
  - st_radlim
  - st_teff
  - st_tefferr1
  - st_tefferr2
  - st_tefflim

🔸 Unique to Kepler (138 columns):
  - kepid
  - kepler_name
  - kepoi_name
  - koi_bin_oedp_sig

## 5. Data Quality Analysis

Examine data quality issues for each dataset:
- Duplicates
- Missing value patterns
- Data type inconsistencies
- Outliers or anomalies

In [18]:
# Check for duplicates
print("=" * 80)
print("DATA QUALITY CHECKS")
print("=" * 80)

print("\n🔍 Duplicate Rows:")
print(f"  - Kepler: {kepler_df.duplicated().sum()} duplicates")
print(f"  - K2: {k2_df.duplicated().sum()} duplicates")
print(f"  - TESS: {tess_df.duplicated().sum()} duplicates")

print("\n📊 Missing Data Summary:")
print(f"  - Kepler: {kepler_df.isnull().sum().sum():,} total missing values")
print(f"  - K2: {k2_df.isnull().sum().sum():,} total missing values")
print(f"  - TESS: {tess_df.isnull().sum().sum():,} total missing values")

print("\n💾 Memory Usage:")
print(f"  - Kepler: {kepler_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  - K2: {k2_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  - TESS: {tess_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

DATA QUALITY CHECKS

🔍 Duplicate Rows:
  - Kepler: 0 duplicates
  - K2: 0 duplicates
  - TESS: 0 duplicates

📊 Missing Data Summary:
  - Kepler: 237,112 total missing values
  - K2: 552,676 total missing values
  - TESS: 111,013 total missing values

💾 Memory Usage:
  - Kepler: 19.73 MB
  - K2: 16.77 MB
  - TESS: 7.10 MB


## 6. Data Cleaning Strategy

Based on the exploration above, we'll implement cleaning for each dataset:

### Key Findings:
- **No duplicate rows** in any dataset ✓
- **High missing values**: Kepler (237K), K2 (552K), TESS (111K)
- **Different schemas**: Very few common columns across all three datasets
- **Different purposes**: Kepler/K2 use KOI data, while K2/TESS use confirmed planet data

### Cleaning Approach:
1. **Remove columns with 100% missing values** (uninformative)
2. **Remove columns with >95% missing values** (likely too sparse)
3. **Standardize column names** (lowercase, underscores)
4. **Identify key columns** for each dataset
5. **Handle remaining missing values** based on column importance

In [19]:
# Function to clean individual datasets
def clean_dataset(df, dataset_name, missing_threshold=95):
    """
    Clean a dataset by removing columns with too many missing values
    
    Parameters:
    - df: DataFrame to clean
    - dataset_name: Name for logging
    - missing_threshold: Percentage threshold for removing columns (default 95%)
    
    Returns:
    - Cleaned DataFrame
    """
    print(f"\n{'='*80}")
    print(f"CLEANING {dataset_name.upper()} DATASET")
    print(f"{'='*80}")
    
    original_shape = df.shape
    print(f"Original shape: {original_shape[0]:,} rows × {original_shape[1]} columns")
    
    # 1. Identify columns with excessive missing values
    missing_pct = (df.isnull().sum() / len(df) * 100)
    cols_to_drop = missing_pct[missing_pct >= missing_threshold].index.tolist()
    
    print(f"\n📊 Columns with ≥{missing_threshold}% missing values: {len(cols_to_drop)}")
    if len(cols_to_drop) > 0 and len(cols_to_drop) <= 20:
        for col in cols_to_drop:
            print(f"   - {col}: {missing_pct[col]:.1f}% missing")
    elif len(cols_to_drop) > 20:
        print(f"   (Showing first 10 of {len(cols_to_drop)})")
        for col in cols_to_drop[:10]:
            print(f"   - {col}: {missing_pct[col]:.1f}% missing")
    
    # 2. Drop columns with excessive missing values
    df_cleaned = df.drop(columns=cols_to_drop)
    print(f"\n✂️  Removed {len(cols_to_drop)} columns with excessive missing values")
    
    # 3. Report remaining missing values
    remaining_missing = df_cleaned.isnull().sum().sum()
    total_values = df_cleaned.shape[0] * df_cleaned.shape[1]
    missing_pct_total = (remaining_missing / total_values * 100)
    
    print(f"\n📉 Remaining missing values: {remaining_missing:,} ({missing_pct_total:.2f}% of total)")
    
    # 4. Show columns with most missing values (that remain)
    remaining_missing_by_col = df_cleaned.isnull().sum()
    top_missing = remaining_missing_by_col[remaining_missing_by_col > 0].sort_values(ascending=False).head(10)
    
    if len(top_missing) > 0:
        print(f"\n🔝 Top columns with missing values (after cleaning):")
        for col, count in top_missing.items():
            pct = (count / len(df_cleaned) * 100)
            print(f"   - {col}: {count:,} ({pct:.1f}%)")
    
    # 5. Final shape
    final_shape = df_cleaned.shape
    print(f"\n✅ Final shape: {final_shape[0]:,} rows × {final_shape[1]} columns")
    print(f"   Reduced from {original_shape[1]} to {final_shape[1]} columns ({original_shape[1] - final_shape[1]} removed)")
    
    return df_cleaned

# Clean each dataset
print("Starting data cleaning process...")
print("Using 95% missing value threshold for column removal\n")

kepler_clean = clean_dataset(kepler_df.copy(), "Kepler", missing_threshold=95)
k2_clean = clean_dataset(k2_df.copy(), "K2", missing_threshold=95)
tess_clean = clean_dataset(tess_df.copy(), "TESS", missing_threshold=95)

print(f"\n{'='*80}")
print("CLEANING SUMMARY")
print(f"{'='*80}")
print(f"✓ Kepler: {kepler_df.shape[1]} → {kepler_clean.shape[1]} columns ({kepler_df.shape[0]:,} rows)")
print(f"✓ K2: {k2_df.shape[1]} → {k2_clean.shape[1]} columns ({k2_df.shape[0]:,} rows)")
print(f"✓ TESS: {tess_df.shape[1]} → {tess_clean.shape[1]} columns ({tess_df.shape[0]:,} rows)")
print(f"\n🎯 All datasets cleaned and ready for further analysis or export!")

Starting data cleaning process...
Using 95% missing value threshold for column removal


CLEANING KEPLER DATASET
Original shape: 9,564 rows × 141 columns

📊 Columns with ≥95% missing values: 19
   - koi_eccen_err1: 100.0% missing
   - koi_eccen_err2: 100.0% missing
   - koi_longp: 100.0% missing
   - koi_longp_err1: 100.0% missing
   - koi_longp_err2: 100.0% missing
   - koi_ingress: 100.0% missing
   - koi_ingress_err1: 100.0% missing
   - koi_ingress_err2: 100.0% missing
   - koi_sma_err1: 100.0% missing
   - koi_sma_err2: 100.0% missing
   - koi_incl_err1: 100.0% missing
   - koi_incl_err2: 100.0% missing
   - koi_teq_err1: 100.0% missing
   - koi_teq_err2: 100.0% missing
   - koi_model_dof: 100.0% missing
   - koi_model_chisq: 100.0% missing
   - koi_sage: 100.0% missing
   - koi_sage_err1: 100.0% missing
   - koi_sage_err2: 100.0% missing

✂️  Removed 19 columns with excessive missing values

📉 Remaining missing values: 55,396 (4.75% of total)

🔝 Top columns with missing values (a

## 7. Dataset Combination Decision

### Analysis of Schema Comparison:

Based on the schema comparison in Section 4, these datasets have **very different structures**:
- **Kepler**: 122 columns (KOI-focused with `koi_` prefixes)
- **K2**: 248 columns (Planet-focused with `pl_` and `st_` prefixes)  
- **TESS**: 75 columns (Planet-focused with `pl_` and `st_` prefixes)

**Common columns across all three**: Very few (likely just metadata like `rowid`)

### Recommended Strategy: **Keep Datasets Separate**

**Rationale:**
1. **Different data models**: Kepler uses KOI (Kepler Object of Interest) data, while K2 and TESS use confirmed planet data
2. **Different missions/timeframes**: Each telescope has unique observation characteristics
3. **Minimal column overlap**: Very few shared columns means combining would create extreme sparsity
4. **Different analysis needs**: Researchers typically analyze these datasets separately by mission

### Alternative: Create Mission Identifier
If you want a unified dataset for specific analyses, we can:
1. Select only the common columns across datasets
2. Add a `mission` column to identify the source
3. Concatenate vertically

Let's keep them separate for now and export as individual cleaned files.

In [20]:
# Decision: Keep datasets separate due to different schemas
# This preserves the unique characteristics and columns of each mission

print("="*80)
print("COMBINATION DECISION")
print("="*80)

print("\n📋 Decision: Keep datasets SEPARATE")
print("\nReasons:")
print("  1. Different data models (KOI vs confirmed planets)")
print("  2. Different missions with unique observation characteristics")
print("  3. Minimal column overlap would create excessive sparsity if combined")
print("  4. Standard practice in exoplanet research is mission-specific analysis")

print("\n✓ Each cleaned dataset will be exported separately")
print("✓ This allows for mission-specific analyses while maintaining data integrity")

# Optional: Show what columns would be common if we wanted to combine later
common_cols = set(kepler_clean.columns) & set(k2_clean.columns) & set(tess_clean.columns)
print(f"\n💡 Note: Only {len(common_cols)} columns are common across all three datasets:")
if common_cols:
    for col in sorted(common_cols):
        print(f"     - {col}")
else:
    print("     (none)")
    
print("\n🔄 If needed, you can create a unified dataset later by selecting common columns")

COMBINATION DECISION

📋 Decision: Keep datasets SEPARATE

Reasons:
  1. Different data models (KOI vs confirmed planets)
  2. Different missions with unique observation characteristics
  3. Minimal column overlap would create excessive sparsity if combined
  4. Standard practice in exoplanet research is mission-specific analysis

✓ Each cleaned dataset will be exported separately
✓ This allows for mission-specific analyses while maintaining data integrity

💡 Note: Only 3 columns are common across all three datasets:
     - dec
     - ra
     - rowid

🔄 If needed, you can create a unified dataset later by selecting common columns


## 8. Export Cleaned Data

Save the cleaned datasets to the `data/clean/` folder.

In [21]:
# Create clean data directory if it doesn't exist
clean_dir = Path('../data/clean')
clean_dir.mkdir(parents=True, exist_ok=True)

print("="*80)
print("EXPORTING CLEANED DATASETS")
print("="*80)

# Export each cleaned dataset
print(f"\n📁 Export directory: {clean_dir.absolute()}\n")

# Save Kepler
kepler_path = clean_dir / 'kepler_clean.csv'
kepler_clean.to_csv(kepler_path, index=False)
print(f"✓ Kepler dataset saved: kepler_clean.csv")
print(f"  - {kepler_clean.shape[0]:,} rows × {kepler_clean.shape[1]} columns")
print(f"  - File size: {kepler_path.stat().st_size / 1024**2:.2f} MB")

# Save K2
k2_path = clean_dir / 'k2_clean.csv'
k2_clean.to_csv(k2_path, index=False)
print(f"\n✓ K2 dataset saved: k2_clean.csv")
print(f"  - {k2_clean.shape[0]:,} rows × {k2_clean.shape[1]} columns")
print(f"  - File size: {k2_path.stat().st_size / 1024**2:.2f} MB")

# Save TESS
tess_path = clean_dir / 'tess_clean.csv'
tess_clean.to_csv(tess_path, index=False)
print(f"\n✓ TESS dataset saved: tess_clean.csv")
print(f"  - {tess_clean.shape[0]:,} rows × {tess_clean.shape[1]} columns")
print(f"  - File size: {tess_path.stat().st_size / 1024**2:.2f} MB")

# Summary
print(f"\n{'='*80}")
print("EXPORT COMPLETE")
print(f"{'='*80}")
print(f"\n🎉 All cleaned datasets have been exported to: {clean_dir.absolute()}")
print(f"\n📊 Summary:")
print(f"   - Total rows exported: {kepler_clean.shape[0] + k2_clean.shape[0] + tess_clean.shape[0]:,}")
print(f"   - Kepler: {kepler_clean.shape[1]} columns (reduced from {kepler_df.shape[1]})")
print(f"   - K2: {k2_clean.shape[1]} columns (reduced from {k2_df.shape[1]})")
print(f"   - TESS: {tess_clean.shape[1]} columns (reduced from {tess_df.shape[1]})")
print(f"\n✅ Data cleaning pipeline complete!")
print(f"   Next steps: Use these cleaned datasets for analysis, visualization, or modeling")

EXPORTING CLEANED DATASETS

📁 Export directory: /Users/jorgesandoval/Documents/current/fermix/notebooks/../data/clean

✓ Kepler dataset saved: kepler_clean.csv
  - 9,564 rows × 122 columns
  - File size: 9.42 MB
✓ Kepler dataset saved: kepler_clean.csv
  - 9,564 rows × 122 columns
  - File size: 9.42 MB

✓ K2 dataset saved: k2_clean.csv
  - 4,004 rows × 248 columns
  - File size: 6.66 MB

✓ TESS dataset saved: tess_clean.csv
  - 7,703 rows × 75 columns
  - File size: 3.47 MB

EXPORT COMPLETE

🎉 All cleaned datasets have been exported to: /Users/jorgesandoval/Documents/current/fermix/notebooks/../data/clean

📊 Summary:
   - Total rows exported: 21,271
   - Kepler: 122 columns (reduced from 141)
   - K2: 248 columns (reduced from 295)
   - TESS: 75 columns (reduced from 87)

✅ Data cleaning pipeline complete!
   Next steps: Use these cleaned datasets for analysis, visualization, or modeling

✓ K2 dataset saved: k2_clean.csv
  - 4,004 rows × 248 columns
  - File size: 6.66 MB

✓ TESS data

## Summary & Results

### ✅ Completed Workflow:
1. **Setup Complete** - Libraries imported and environment configured
2. **Data Loaded** - Successfully loaded three NASA exoplanet datasets
3. **Exploration Complete** - Examined structure, types, and quality of each dataset
4. **Schema Compared** - Identified only 3 common columns (ra, dec, rowid) across all datasets
5. **Data Cleaned** - Removed columns with ≥95% missing values
6. **Decision Made** - Keep datasets separate due to different missions and data models
7. **Export Complete** - Saved cleaned datasets to `data/clean/` folder

### 📊 Final Results:

| Dataset | Original | Cleaned | Rows | File Size |
|---------|----------|---------|------|-----------|
| **Kepler** | 141 cols | 122 cols | 9,564 | 9.42 MB |
| **K2** | 295 cols | 248 cols | 4,004 | 6.66 MB |
| **TESS** | 87 cols | 75 cols | 7,703 | 3.47 MB |

### 🎯 Key Findings:
- **No duplicates** found in any dataset
- Removed **78 total columns** with excessive missing values (≥95%)
- **Different data models**: Kepler uses KOI data, K2/TESS use confirmed planet data
- **Minimal overlap**: Only 3 columns common across all datasets
- **Kept separate**: Best practice for mission-specific exoplanet research

### 📁 Output Files:
All cleaned datasets are saved in `data/clean/`:
- `kepler_clean.csv`
- `k2_clean.csv`
- `tess_clean.csv`

### 🚀 Next Steps:
- Perform mission-specific analysis on each dataset
- Create visualizations of planet properties
- Build predictive models
- Compare discovery methods across missions

---
*Data cleaning pipeline completed successfully! The cleaned datasets are ready for analysis.*